## Polymarket API Reference

https://colab.research.google.com/drive/13mGFq_BqskRUxxIZoimomaD_6cKFfWnF#scrollTo=oinNv2Y9x-pO

In this phase of our research, we would be using the **Gamma API**, which is a REST API, a web endpoint. We're exploring the various bets or contracts that are available. And we can think about what we know about them, are there data sources we can explore or anything we can think of to try to beat the odds. Once we have explored these, we can use another API to see what other people are doing in real time. How are these Yes / No values moving second by second. During the speech, when does a value start spiking or plummeting.

# *Extract Data*

#### Template 1.0 : Request method

In [103]:
import requests
import pandas as pd
from datetime import datetime, timezone


#url = "https://clob.polymarket.com/prices-history?interval=1w&market=74500135454074136715064885530486995983445722288714292304246782362541004861272&fidelity=15"

# Hit $110,000
url = "https://clob.polymarket.com/prices-history?interval=1w&market=74500135454074136715064885530486995983445722288714292304246782362541004861272&fidelity=15"

response = requests.get(url)

print("Checking response......")
if response.status_code!=200:
    print(f"Non-success status code: {response.status_code}")
else:
    print(response)

print("\nChecking modified json......")
data1 = response.json()['history']
print(data1)


print("\nTransforming data......")
df1 = pd.DataFrame(data1)
df1['t'] = pd.to_datetime(df1['t'],unit='s')
df1



Checking response......
<Response [200]>

Checking modified json......
[{'t': 1738355404, 'p': 0.5}, {'t': 1738356304, 'p': 0.48}, {'t': 1738357204, 'p': 0.47}, {'t': 1738358104, 'p': 0.48}, {'t': 1738359004, 'p': 0.48}, {'t': 1738359904, 'p': 0.495}, {'t': 1738360804, 'p': 0.495}, {'t': 1738361703, 'p': 0.495}, {'t': 1738362604, 'p': 0.495}, {'t': 1738363504, 'p': 0.405}, {'t': 1738364404, 'p': 0.41}, {'t': 1738365304, 'p': 0.39}, {'t': 1738366204, 'p': 0.51}, {'t': 1738367104, 'p': 0.51}, {'t': 1738368005, 'p': 0.51}, {'t': 1738368904, 'p': 0.51}, {'t': 1738369805, 'p': 0.51}, {'t': 1738370704, 'p': 0.51}, {'t': 1738371604, 'p': 0.53}, {'t': 1738372504, 'p': 0.53}, {'t': 1738373404, 'p': 0.505}, {'t': 1738374304, 'p': 0.525}, {'t': 1738375204, 'p': 0.505}, {'t': 1738376104, 'p': 0.505}, {'t': 1738377004, 'p': 0.505}, {'t': 1738377904, 'p': 0.505}, {'t': 1738378804, 'p': 0.505}, {'t': 1738379703, 'p': 0.545}, {'t': 1738380604, 'p': 0.505}, {'t': 1738381505, 'p': 0.505}, {'t': 17383824

,t,p
0,2025-01-31 20:30:04,0.500
1,2025-01-31 20:45:04,0.480
2,2025-01-31 21:00:04,0.470
3,2025-01-31 21:15:04,0.480
4,2025-01-31 21:30:04,0.480
...,...,...
270,2025-02-03 16:00:04,0.450
271,2025-02-03 16:15:04,0.440
272,2025-02-03 16:30:04,0.450
273,2025-02-03 16:45:04,0.440


#### Template 2.0: Work on CSV Dataframes

In [104]:
#specific filepath

file_path_nov = "./price_odds_data/polymarket-price-data-03-11-2024-01-12-2024-1738593213639.csv"
file_path_dec = "./price_odds_data/polymarket-price-data-02-12-2024-01-01-2025-1738593154283.csv"
file_path_jan = "./price_odds_data/polymarket-price-data-02-01-2025-01-02-2025-1738593110866.csv"
file_path_feb = "./price_odds_data/polymarket-price-data-31-01-2025-03-02-2025-1738593057495.csv"

 
df_jan = pd.read_csv(file_path_jan)
df_feb = pd.read_csv(file_path_feb)
df_dec = pd.read_csv(file_path_dec)
df_nov = pd.read_csv(file_path_nov)


In [105]:
def find_first_row(df, column_index):
    # Convert the specified column to numeric, replacing non-numeric values with NaN
    column_name = df.columns[column_index]  # Get column name from index
    numeric_series = pd.to_numeric(df[column_name], errors='coerce')
    
    # Find the first valid index (row) where a number appears
    first_row = numeric_series.first_valid_index()
    
    return first_row

def find_first_row_num(df):
    row_num = dict()
    for i in range (2,len(df.columns)):
        row_num[df.columns[i]] = find_first_row(df,i)
        
    return row_num

#testing
find_first_row_num(df_feb)


{'$200,000': 0,
 '$150,000': 0,
 '$140,000': 0,
 '$130,000': 0,
 '$120,000': 0,
 '$115,000': 0,
 '$110,000': 0,
 '$90,000': 0,
 '$85,000': 0,
 '$75,000': 0,
 '$60,000': 0,
 '$100,000': 0,
 '$95,000': 0}

In [107]:
def fill_nan_after_each_row(df, row_indices):
    """
    Fill NaN values with 1 in specified columns after their respective row indices.

    Parameters:
        df (pd.DataFrame): The input DataFrame.
        row_indices (dict): A dictionary where keys are column names and values are row indices.

    Returns:
        pd.DataFrame: The modified DataFrame.
    """
    
    for column, row_index in row_indices.items():
        
        # Create a mask for rows after the specified row_index for the column
        mask = df.index > row_index
        # Fill NaN values with 1 only for that column after its specific row_index
        df.loc[mask, column] = df.loc[mask, column].fillna(1)

    return df


#### Cleansing on Feb Data

In [106]:
#Before edit
df_feb

,Date (UTC),Timestamp (UTC),"$200,000","$150,000","$140,000","$130,000","$120,000","$115,000","$110,000","$90,000","$85,000","$75,000","$60,000","$100,000","$95,000"
0,01-31-2025 21:00,1738357204,0.0550,0.0500,0.0750,0.190,0.245,0.420,0.470,0.520,0.495,0.1750,0.0500,0.825,0.500
1,01-31-2025 22:00,1738360804,0.0300,0.0400,0.0650,0.190,0.245,0.430,0.495,0.520,0.500,0.1800,0.0500,0.850,0.475
2,01-31-2025 23:00,1738364404,0.0300,0.0400,0.0550,0.195,0.250,0.345,0.410,0.360,0.500,0.1700,0.0450,0.850,0.475
3,02-01-2025 00:00,1738368005,0.0295,0.0500,0.0650,0.175,0.270,0.375,0.510,0.360,0.500,0.1000,0.0400,0.855,0.500
4,02-01-2025 01:00,1738371604,0.0275,0.0500,0.0650,0.170,0.265,0.350,0.530,0.370,0.500,0.0950,0.0450,0.800,0.450
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62,02-03-2025 11:00,1738580404,0.0095,0.0215,0.0310,0.065,0.155,0.220,0.350,0.615,0.305,0.0835,0.0335,NaN,NaN
63,02-03-2025 12:00,1738584004,0.0085,0.0220,0.0310,0.065,0.130,0.210,0.320,0.615,0.335,0.0845,0.0370,NaN,NaN
64,02-03-2025 13:00,1738587604,0.0085,0.0215,0.0305,0.065,0.120,0.195,0.315,0.615,0.340,0.0860,0.0335,NaN,NaN
65,02-03-2025 14:00,1738591203,0.0090,0.0220,0.0305,0.060,0.120,0.190,0.315,0.655,0.355,0.0795,0.0285,NaN,NaN


In [144]:
df_feb.tail(10)

,Date (UTC),Timestamp (UTC),"$200,000","$150,000","$140,000","$130,000","$120,000","$115,000","$110,000","$90,000","$85,000","$75,000","$60,000","$100,000","$95,000"
57,02-03-2025 06:00,1738562404,0.0075,0.0240,0.0320,0.055,0.110,0.190,0.295,0.745,0.415,0.1170,0.0450,NaN,NaN
58,02-03-2025 07:00,1738566004,0.0075,0.0205,0.0265,0.055,0.130,0.185,0.300,0.760,0.420,0.0970,0.0425,NaN,NaN
59,02-03-2025 08:00,1738569604,0.0075,0.0195,0.0270,0.065,0.135,0.200,0.360,0.665,0.350,0.0870,0.0415,NaN,NaN
60,02-03-2025 09:00,1738573204,0.0075,0.0200,0.0300,0.065,0.140,0.205,0.355,0.645,0.325,0.0870,0.0355,NaN,NaN
61,02-03-2025 10:00,1738576804,0.0080,0.0220,0.0305,0.065,0.155,0.225,0.365,0.625,0.315,0.0850,0.0340,NaN,NaN
62,02-03-2025 11:00,1738580404,0.0095,0.0215,0.0310,0.065,0.155,0.220,0.350,0.615,0.305,0.0835,0.0335,NaN,NaN
63,02-03-2025 12:00,1738584004,0.0085,0.0220,0.0310,0.065,0.130,0.210,0.320,0.615,0.335,0.0845,0.0370,NaN,NaN
64,02-03-2025 13:00,1738587604,0.0085,0.0215,0.0305,0.065,0.120,0.195,0.315,0.615,0.340,0.0860,0.0335,NaN,NaN
65,02-03-2025 14:00,1738591203,0.0090,0.0220,0.0305,0.060,0.120,0.190,0.315,0.655,0.355,0.0795,0.0285,NaN,NaN
66,02-03-2025 14:30,1738593004,0.0090,0.0225,0.0305,0.055,0.115,0.190,0.285,0.665,0.380,0.0815,0.0290,NaN,NaN


***** Edit Feb data ***** 

In [145]:
row_indices = find_first_row_num(df_feb)
df_feb_edit1 = df_feb.copy(deep=True)
df_feb_edit1 = fill_nan_after_each_row(df_feb_edit1, row_indices)
df_feb_edit1

,Date (UTC),Timestamp (UTC),"$200,000","$150,000","$140,000","$130,000","$120,000","$115,000","$110,000","$90,000","$85,000","$75,000","$60,000","$100,000","$95,000"
0,01-31-2025 21:00,1738357204,0.0550,0.0500,0.0750,0.190,0.245,0.420,0.470,0.520,0.495,0.1750,0.0500,0.825,0.500
1,01-31-2025 22:00,1738360804,0.0300,0.0400,0.0650,0.190,0.245,0.430,0.495,0.520,0.500,0.1800,0.0500,0.850,0.475
2,01-31-2025 23:00,1738364404,0.0300,0.0400,0.0550,0.195,0.250,0.345,0.410,0.360,0.500,0.1700,0.0450,0.850,0.475
3,02-01-2025 00:00,1738368005,0.0295,0.0500,0.0650,0.175,0.270,0.375,0.510,0.360,0.500,0.1000,0.0400,0.855,0.500
4,02-01-2025 01:00,1738371604,0.0275,0.0500,0.0650,0.170,0.265,0.350,0.530,0.370,0.500,0.0950,0.0450,0.800,0.450
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62,02-03-2025 11:00,1738580404,0.0095,0.0215,0.0310,0.065,0.155,0.220,0.350,0.615,0.305,0.0835,0.0335,1.000,1.000
63,02-03-2025 12:00,1738584004,0.0085,0.0220,0.0310,0.065,0.130,0.210,0.320,0.615,0.335,0.0845,0.0370,1.000,1.000
64,02-03-2025 13:00,1738587604,0.0085,0.0215,0.0305,0.065,0.120,0.195,0.315,0.615,0.340,0.0860,0.0335,1.000,1.000
65,02-03-2025 14:00,1738591203,0.0090,0.0220,0.0305,0.060,0.120,0.190,0.315,0.655,0.355,0.0795,0.0285,1.000,1.000


#### Cleansing on Jan Data

In [110]:
df_jan.tail(10)

,Date (UTC),Timestamp (UTC),"$200,000","$150,000","$140,000","$130,000","$120,000","$110,000","$105,000","$100,000","$90,000","$85,000","$80,000","$70,000","$60,000","$50,000"
699,01-31-2025 23:00,1738364404,0.0005,0.0005,0.0005,0.0005,0.0005,0.0025,NaN,NaN,NaN,0.0005,0.0015,0.0010,0.0010,0.0010
700,02-01-2025 00:00,1738368005,0.0005,0.0005,0.0015,0.0005,0.0010,0.0015,NaN,NaN,NaN,0.0005,0.0010,0.0010,0.0010,0.0010
701,02-01-2025 01:00,1738371604,0.0005,0.0005,0.0005,0.0005,0.0010,0.0020,NaN,NaN,NaN,0.0005,0.0010,0.0010,0.0010,0.0010
702,02-01-2025 02:00,1738375204,0.0005,0.0005,0.0005,0.0005,0.0005,0.0015,NaN,NaN,NaN,0.0005,0.0010,0.0005,0.0005,0.0005
703,02-01-2025 03:00,1738378804,0.0005,0.0005,0.0005,0.0005,0.0010,0.0005,NaN,NaN,NaN,0.0005,0.0010,0.0005,0.0005,0.0005
704,02-01-2025 04:00,1738382404,0.0005,0.0005,0.0005,0.0005,0.0005,0.0005,NaN,NaN,NaN,0.0005,0.0005,0.0005,0.0005,0.0005
705,02-01-2025 05:00,1738386004,0.0005,0.0005,0.0005,0.0005,0.0005,0.0005,NaN,NaN,NaN,0.0005,0.0005,0.0005,0.0005,0.0005
706,02-01-2025 06:00,1738389604,0.0005,0.0005,0.0005,0.0005,0.0005,0.0005,NaN,NaN,NaN,0.0005,0.0005,0.0005,0.0005,0.0005
707,02-01-2025 07:00,1738393204,0.0005,0.0005,0.0005,0.0005,0.0005,0.0005,NaN,NaN,NaN,0.0005,0.0005,0.0005,0.0005,0.0005
708,02-01-2025 08:00,1738396804,NaN,NaN,NaN,0.0005,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [111]:
row_indices = find_first_row_num(df_jan)
df_jan_edit1 = df_jan.copy(deep=True)
df_jan_edit1 = fill_nan_after_each_row(df_jan_edit1, row_indices)
df_jan_edit1 = df_jan_edit1.iloc[:-1]

df_jan_edit1

,Date (UTC),Timestamp (UTC),"$200,000","$150,000","$140,000","$130,000","$120,000","$110,000","$105,000","$100,000","$90,000","$85,000","$80,000","$70,000","$60,000","$50,000"
0,01-02-2025 20:00,1735848003,0.0250,0.0700,0.0700,0.1600,0.3900,0.4750,0.790,0.840,0.450,0.1300,0.2500,0.0950,0.1000,0.0800
1,01-02-2025 21:00,1735851603,0.0320,0.0300,0.0400,0.1000,0.3050,0.4650,0.640,0.840,0.590,0.1900,0.2500,0.0900,0.0900,0.0550
2,01-02-2025 22:00,1735855203,0.0275,0.0300,0.0395,0.0900,0.2100,0.4150,0.585,0.845,0.595,0.3800,0.2800,0.0900,0.0900,0.0350
3,01-02-2025 23:00,1735858804,0.0125,0.0295,0.0390,0.0700,0.1500,0.3900,0.580,0.845,0.590,0.3650,0.2500,0.0950,0.0350,0.0125
4,01-03-2025 00:00,1735862404,0.0125,0.0275,0.0390,0.0700,0.1650,0.3950,0.580,0.850,0.590,0.3950,0.2400,0.0650,0.0255,0.0125
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
703,02-01-2025 03:00,1738378804,0.0005,0.0005,0.0005,0.0005,0.0010,0.0005,1.000,1.000,1.000,0.0005,0.0010,0.0005,0.0005,0.0005
704,02-01-2025 04:00,1738382404,0.0005,0.0005,0.0005,0.0005,0.0005,0.0005,1.000,1.000,1.000,0.0005,0.0005,0.0005,0.0005,0.0005
705,02-01-2025 05:00,1738386004,0.0005,0.0005,0.0005,0.0005,0.0005,0.0005,1.000,1.000,1.000,0.0005,0.0005,0.0005,0.0005,0.0005
706,02-01-2025 06:00,1738389604,0.0005,0.0005,0.0005,0.0005,0.0005,0.0005,1.000,1.000,1.000,0.0005,0.0005,0.0005,0.0005,0.0005


#### Cleansing on Dec Data

In [112]:
df_dec.tail(20)

,Date (UTC),Timestamp (UTC),$130k,$120k,$110k,$105k,$100k,$90k,$85k
691,12-31-2024 12:00,1735646403,0.0015,0.0005,0.0025,NaN,NaN,0.0450,0.0055
692,12-31-2024 13:00,1735650003,0.0015,0.0005,0.0030,NaN,NaN,0.0430,0.0040
693,12-31-2024 14:00,1735653603,0.0010,0.0025,0.0035,NaN,NaN,0.0165,0.0030
694,12-31-2024 15:00,1735657203,0.0015,0.0025,0.0020,NaN,NaN,0.0115,0.0035
695,12-31-2024 16:00,1735660803,0.0015,0.0005,0.0015,NaN,NaN,0.0095,0.0015
696,12-31-2024 17:00,1735664404,0.0010,0.0005,0.0015,NaN,NaN,0.0140,0.0025
697,12-31-2024 18:00,1735668004,0.0010,0.0005,0.0025,NaN,NaN,0.0155,0.0035
698,12-31-2024 19:00,1735671603,0.0010,0.0005,0.0025,NaN,NaN,0.0215,0.0040
699,12-31-2024 20:00,1735675204,0.0010,0.0015,0.0030,NaN,NaN,0.0150,0.0020
700,12-31-2024 21:00,1735678804,0.0010,0.0015,0.0025,NaN,NaN,0.0190,0.0015


In [113]:
row_indices = find_first_row_num(df_dec)
df_dec_edit1 = df_dec.copy(deep=True)
df_dec_edit1 = fill_nan_after_each_row(df_dec_edit1, row_indices)

In [114]:
df_dec_edit1

,Date (UTC),Timestamp (UTC),$130k,$120k,$110k,$105k,$100k,$90k,$85k
0,12-02-2024 17:00,1733158803,0.0850,0.1200,0.1550,0.430,0.695,0.4750,0.4450
1,12-02-2024 18:00,1733162402,0.0850,0.1450,0.2800,0.435,0.660,0.6600,0.3900
2,12-02-2024 19:00,1733166003,0.0950,0.1450,0.2900,0.435,0.685,0.5750,0.3450
3,12-02-2024 20:00,1733169603,0.0750,0.1450,0.2900,0.425,0.670,0.5450,0.3650
4,12-02-2024 21:00,1733173203,0.0850,0.1650,0.3050,0.480,0.700,0.5550,0.3350
...,...,...,...,...,...,...,...,...,...
706,01-01-2025 03:00,1735700404,0.0005,0.0005,0.0005,1.000,1.000,0.0015,0.0005
707,01-01-2025 04:00,1735704003,0.0005,0.0005,0.0005,1.000,1.000,0.0015,0.0005
708,01-01-2025 05:00,1735707603,0.0005,0.0005,0.0005,1.000,1.000,0.0005,0.0005
709,01-01-2025 06:00,1735711204,0.0005,0.0005,0.0005,1.000,1.000,0.0005,0.0005


#### Cleansing on Nov Data

In [115]:
df_nov.tail(20)

,Date (UTC),Timestamp (UTC),"$150,000","$140,000","$130,000","$120,000","$110,000","$105,000","$95,000","$90,000","$87,500","$85,000","$82,500","$80,000","$77,500","$75,000","$72,500","$67,500","$65,000","$60,000"
659,11-30-2024 12:00,1732968003,0.0015,0.0005,0.0015,0.0015,0.0040,0.0065,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0025,0.0015
660,11-30-2024 13:00,1732971603,0.0010,0.0010,0.0005,0.0015,0.0035,0.0070,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0025,0.0015
661,11-30-2024 14:00,1732975203,0.0015,0.0020,0.0010,0.0015,0.0030,0.0045,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0020,0.0015
662,11-30-2024 15:00,1732978803,0.0010,0.0015,0.0015,0.0015,0.0030,0.0040,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0025,0.0015
663,11-30-2024 16:00,1732982404,0.0010,0.0005,0.0015,0.0015,0.0045,0.0035,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0025,0.0020
664,11-30-2024 17:00,1732986003,0.0010,0.0005,0.0010,0.0015,0.0040,0.0030,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0025,0.0020
665,11-30-2024 18:00,1732989603,0.0010,0.0005,0.0010,0.0015,0.0035,0.0045,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0025,0.0020
666,11-30-2024 19:00,1732993202,0.0010,0.0005,0.0010,0.0030,0.0025,0.0035,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0025,0.0020
667,11-30-2024 20:00,1732996803,0.0010,0.0010,0.0010,0.0020,0.0025,0.0045,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0025,0.0020
668,11-30-2024 21:00,1733000403,0.0010,0.0010,0.0010,0.0020,0.0025,0.0025,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0025,0.0020


In [116]:
row_indices = find_first_row_num(df_nov)
df_nov_edit1 = df_nov.copy(deep=True)
df_nov_edit1 = fill_nan_after_each_row(df_nov_edit1, row_indices)

In [117]:
df_nov_edit1

,Date (UTC),Timestamp (UTC),"$150,000","$140,000","$130,000","$120,000","$110,000","$105,000","$95,000","$90,000","$87,500","$85,000","$82,500","$80,000","$77,500","$75,000","$72,500","$67,500","$65,000","$60,000"
0,11-03-2024 00:00,1730592003,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.130,NaN,0.225,NaN,0.375,0.505,0.645,0.785,0.830,0.6100,0.2850
1,11-03-2024 01:00,1730595603,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.130,NaN,0.230,NaN,0.375,0.510,0.640,0.785,0.830,0.6100,0.2850
2,11-03-2024 02:00,1730599203,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.125,NaN,0.225,NaN,0.360,0.505,0.635,0.780,0.845,0.6150,0.2950
3,11-03-2024 03:00,1730602803,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.120,NaN,0.235,NaN,0.335,0.455,0.580,0.695,0.870,0.6550,0.3450
4,11-03-2024 04:00,1730606403,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.120,NaN,0.235,NaN,0.340,0.460,0.580,0.725,0.855,0.6450,0.3400
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
674,12-01-2024 03:00,1733022003,0.0010,0.0005,0.0005,0.0005,0.0010,0.0005,1.0,1.000,1.0,1.000,1.0,1.000,1.000,1.000,1.000,1.000,0.0015,0.0015
675,12-01-2024 04:00,1733025603,0.0010,0.0005,0.0005,0.0020,0.0010,0.0005,1.0,1.000,1.0,1.000,1.0,1.000,1.000,1.000,1.000,1.000,0.0005,0.0015
676,12-01-2024 05:00,1733029203,0.0005,0.0005,0.0005,0.0005,0.0010,0.0005,1.0,1.000,1.0,1.000,1.0,1.000,1.000,1.000,1.000,1.000,0.0020,0.0005
677,12-01-2024 06:00,1733032803,0.0005,0.0005,0.0005,0.0005,0.0005,0.0005,1.0,1.000,1.0,1.000,1.0,1.000,1.000,1.000,1.000,1.000,0.0005,0.0005


### Export spearate csv

In [120]:
df_nov_edit1.to_csv("priceodds_nov.csv", index=False, sep=';')
df_dec_edit1.to_csv("priceodds_dec.csv", index=False, sep=';')
df_jan_edit1.to_csv("priceodds_jan.csv", index=False, sep=';')
df_feb_edit1.to_csv("priceodds_feb.csv", index=False, sep=';')

# _Consolidate csv_

#### Tidy up Dec - Change header

In [125]:
df_dec_edit1.columns
#df_jan_edit1.columns

Index(['Date (UTC)', 'Timestamp (UTC)', '$130k', '$120k', '$110k', '$105k',
       '$100k', '$90k', '$85k'],
      dtype='object')

In [127]:
df_dec_edit1 = df_dec_edit1.rename(columns={col: col.replace("k", ",000") for col in df_dec_edit1.columns})

df_dec_edit1

,Date (UTC),Timestamp (UTC),"$130,000","$120,000","$110,000","$105,000","$100,000","$90,000","$85,000"
0,12-02-2024 17:00,1733158803,0.0850,0.1200,0.1550,0.430,0.695,0.4750,0.4450
1,12-02-2024 18:00,1733162402,0.0850,0.1450,0.2800,0.435,0.660,0.6600,0.3900
2,12-02-2024 19:00,1733166003,0.0950,0.1450,0.2900,0.435,0.685,0.5750,0.3450
3,12-02-2024 20:00,1733169603,0.0750,0.1450,0.2900,0.425,0.670,0.5450,0.3650
4,12-02-2024 21:00,1733173203,0.0850,0.1650,0.3050,0.480,0.700,0.5550,0.3350
...,...,...,...,...,...,...,...,...,...
706,01-01-2025 03:00,1735700404,0.0005,0.0005,0.0005,1.000,1.000,0.0015,0.0005
707,01-01-2025 04:00,1735704003,0.0005,0.0005,0.0005,1.000,1.000,0.0015,0.0005
708,01-01-2025 05:00,1735707603,0.0005,0.0005,0.0005,1.000,1.000,0.0005,0.0005
709,01-01-2025 06:00,1735711204,0.0005,0.0005,0.0005,1.000,1.000,0.0005,0.0005


#### Combine Dataframe Functions

In [128]:

def combine_dataframes(df1, df2, timestamp_col='Timestamp (UTC)'):
    """
    Combine two dataframes such that for rows with duplicate timestamps,
    the row from df2 is retained. All columns are preserved, with missing
    columns filled as NaN.
    
    Parameters:
    -----------
    df1 : pandas.DataFrame
        The first dataframe.
    df2 : pandas.DataFrame
        The second dataframe. Rows from this dataframe will override those in df1
        when a duplicate timestamp is found.
    timestamp_col : str, optional
        The name of the column containing the timestamp (default is 'timestamp').
    
    Returns:
    --------
    pandas.DataFrame
        A new dataframe combining df1 and df2 with df2 overriding duplicates.
    """
    # Concatenate the two dataframes. Since df2 is placed after df1,
    # any duplicate timestamps will have the df2 entry last.
    combined_df = pd.concat([df1, df2], ignore_index=True, sort=False)
    
    # Drop duplicates based on the timestamp column,
    # keeping the last occurrence (from df2 when duplicates exist)
    combined_df = combined_df.drop_duplicates(subset=timestamp_col, keep='last')
    
    return combined_df

In [130]:
def sort_dataframe_columns(df):
    """
    Sort the DataFrame columns in descending order based on the numeric value
    extracted from their header names, except for the first two columns which remain unchanged.
    
    Parameters:
    -----------
    df : pandas.DataFrame
        DataFrame where the first two columns are 'date(UTC)' and 'timestamp(utc)' and
        the remaining columns have header names formatted like '$110,000', '$130,000', etc.
    
    Returns:
    --------
    pandas.DataFrame
        A new DataFrame with the first two columns unchanged and the rest sorted
        from largest to smallest based on their numeric values.
    """
    # Keep the first two columns unchanged.
    fixed_cols = list(df.columns[:2])
    # The rest of the columns will be sorted.
    other_cols = list(df.columns[2:])
    
    def parse_numeric(col_name):
        """
        Extract the numeric value from a column name.
        Example: '$110,000' -> 110000.0
        """
        # Remove the '$' and ',' characters and convert the result to float.
        numeric_str = col_name.replace('$', '').replace(',', '')
        return float(numeric_str)
    
    # Sort the other columns by the numeric value in descending order.
    sorted_other_cols = sorted(other_cols, key=parse_numeric, reverse=True)
    
    # Combine the fixed columns with the sorted columns.
    new_order = fixed_cols + sorted_other_cols
    return df[new_order]

#### Combine Dataframes Workflow

Combine Jan and Feb

In [134]:
df_jan_feb = combine_dataframes(df_jan_edit1, df_feb_edit1)
df_jan_feb = sort_dataframe_columns(df_jan_feb)
df_jan_feb

,Date (UTC),Timestamp (UTC),"$200,000","$150,000","$140,000","$130,000","$120,000","$115,000","$110,000","$105,000","$100,000","$95,000","$90,000","$85,000","$80,000","$75,000","$70,000","$60,000","$50,000"
0,01-02-2025 20:00,1735848003,0.0250,0.0700,0.0700,0.160,0.390,NaN,0.475,0.790,0.840,NaN,0.450,0.130,0.25,NaN,0.095,0.1000,0.0800
1,01-02-2025 21:00,1735851603,0.0320,0.0300,0.0400,0.100,0.305,NaN,0.465,0.640,0.840,NaN,0.590,0.190,0.25,NaN,0.090,0.0900,0.0550
2,01-02-2025 22:00,1735855203,0.0275,0.0300,0.0395,0.090,0.210,NaN,0.415,0.585,0.845,NaN,0.595,0.380,0.28,NaN,0.090,0.0900,0.0350
3,01-02-2025 23:00,1735858804,0.0125,0.0295,0.0390,0.070,0.150,NaN,0.390,0.580,0.845,NaN,0.590,0.365,0.25,NaN,0.095,0.0350,0.0125
4,01-03-2025 00:00,1735862404,0.0125,0.0275,0.0390,0.070,0.165,NaN,0.395,0.580,0.850,NaN,0.590,0.395,0.24,NaN,0.065,0.0255,0.0125
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
770,02-03-2025 11:00,1738580404,0.0095,0.0215,0.0310,0.065,0.155,0.220,0.350,NaN,1.000,1.0,0.615,0.305,NaN,0.0835,NaN,0.0335,NaN
771,02-03-2025 12:00,1738584004,0.0085,0.0220,0.0310,0.065,0.130,0.210,0.320,NaN,1.000,1.0,0.615,0.335,NaN,0.0845,NaN,0.0370,NaN
772,02-03-2025 13:00,1738587604,0.0085,0.0215,0.0305,0.065,0.120,0.195,0.315,NaN,1.000,1.0,0.615,0.340,NaN,0.0860,NaN,0.0335,NaN
773,02-03-2025 14:00,1738591203,0.0090,0.0220,0.0305,0.060,0.120,0.190,0.315,NaN,1.000,1.0,0.655,0.355,NaN,0.0795,NaN,0.0285,NaN


Combine Nov and Dec

In [135]:
df_nov_dec = combine_dataframes(df_nov_edit1, df_dec_edit1)
df_nov_dec = sort_dataframe_columns(df_nov_dec)
df_nov_dec


,Date (UTC),Timestamp (UTC),"$150,000","$140,000","$130,000","$120,000","$110,000","$105,000","$100,000","$95,000",...,"$87,500","$85,000","$82,500","$80,000","$77,500","$75,000","$72,500","$67,500","$65,000","$60,000"
0,11-03-2024 00:00,1730592003,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,0.2250,NaN,0.375,0.505,0.645,0.785,0.830,0.610,0.285
1,11-03-2024 01:00,1730595603,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,0.2300,NaN,0.375,0.510,0.640,0.785,0.830,0.610,0.285
2,11-03-2024 02:00,1730599203,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,0.2250,NaN,0.360,0.505,0.635,0.780,0.845,0.615,0.295
3,11-03-2024 03:00,1730602803,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,0.2350,NaN,0.335,0.455,0.580,0.695,0.870,0.655,0.345
4,11-03-2024 04:00,1730606403,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,0.2350,NaN,0.340,0.460,0.580,0.725,0.855,0.645,0.340
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1385,01-01-2025 03:00,1735700404,NaN,NaN,0.0005,0.0005,0.0005,1.0,1.0,NaN,...,NaN,0.0005,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1386,01-01-2025 04:00,1735704003,NaN,NaN,0.0005,0.0005,0.0005,1.0,1.0,NaN,...,NaN,0.0005,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1387,01-01-2025 05:00,1735707603,NaN,NaN,0.0005,0.0005,0.0005,1.0,1.0,NaN,...,NaN,0.0005,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1388,01-01-2025 06:00,1735711204,NaN,NaN,0.0005,0.0005,0.0005,1.0,1.0,NaN,...,NaN,0.0005,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Finalising - Combine 4 months

In [137]:
df_nov_feb = combine_dataframes(df_nov_dec, df_jan_feb)
df_nov_feb = sort_dataframe_columns(df_nov_feb)
df_nov_feb

,Date (UTC),Timestamp (UTC),"$200,000","$150,000","$140,000","$130,000","$120,000","$115,000","$110,000","$105,000",...,"$82,500","$80,000","$77,500","$75,000","$72,500","$70,000","$67,500","$65,000","$60,000","$50,000"
0,11-03-2024 00:00,1730592003,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,0.375,0.505,0.6450,0.785,NaN,0.830,0.610,0.2850,NaN
1,11-03-2024 01:00,1730595603,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,0.375,0.510,0.6400,0.785,NaN,0.830,0.610,0.2850,NaN
2,11-03-2024 02:00,1730599203,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,0.360,0.505,0.6350,0.780,NaN,0.845,0.615,0.2950,NaN
3,11-03-2024 03:00,1730602803,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,0.335,0.455,0.5800,0.695,NaN,0.870,0.655,0.3450,NaN
4,11-03-2024 04:00,1730606403,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,0.340,0.460,0.5800,0.725,NaN,0.855,0.645,0.3400,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2149,02-03-2025 11:00,1738580404,0.0095,0.0215,0.0310,0.065,0.155,0.220,0.350,NaN,...,NaN,NaN,NaN,0.0835,NaN,NaN,NaN,NaN,0.0335,NaN
2150,02-03-2025 12:00,1738584004,0.0085,0.0220,0.0310,0.065,0.130,0.210,0.320,NaN,...,NaN,NaN,NaN,0.0845,NaN,NaN,NaN,NaN,0.0370,NaN
2151,02-03-2025 13:00,1738587604,0.0085,0.0215,0.0305,0.065,0.120,0.195,0.315,NaN,...,NaN,NaN,NaN,0.0860,NaN,NaN,NaN,NaN,0.0335,NaN
2152,02-03-2025 14:00,1738591203,0.0090,0.0220,0.0305,0.060,0.120,0.190,0.315,NaN,...,NaN,NaN,NaN,0.0795,NaN,NaN,NaN,NaN,0.0285,NaN


#### Export the Combined Dataframe

In [138]:
df_nov_feb.to_csv("priceodds_combined_nov_feb.csv", index=False, sep=';')

-------------

### Extract Data - Draft